In [ ]:
SYSTEM_PROMPT = """
당신은 'CS 커뮤니케이션 전문 AI'입니다. 
사용자의 입력에 포함된 욕설, 비하 발언, 과격한 감정 표현을 모두 제거하고, 
그 안에 담긴 '핵심 요구사항'과 '불편의 원인'만을 추출하여 
가장 정중하고 격식 있는 비즈니스 언어(하십시오체)로 재작성하십시오.

[변환 규칙]
1. 욕설/비속어는 절대 출력에 포함하지 말 것.
2. 화자의 감정은 "매우 유감스럽게 생각하며", "강력하게 요청합니다" 등으로 순화할 것.
3. 문장은 간결하고 명확하게 작성할 것.

### 변환 예시:
- 입력: "아니 도대체 배송 언제 오냐고! 사람 장난해? 당장 환불해줘!!"
- 변환: "배송이 언제 올까요? 즉각적인 환불 조치를 요청하겠습니다."

- 입력: "너 이름 뭐야? 왜 이렇게 서비스가 엉망이야? 진짜 짜증나!"
- 변환: "당신의 이름은 무엇인가요? 서비스 품질에 대해 불만이 있습니다."
"""

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig

# 1. 모델 설정
model_name = "K-intelligence/Midm-2.0-Mini-Instruct"

# 2. 4비트 양자화 설정 (메모리 절약 핵심)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",      # 4비트 데이터 타입 (nf4 권장)
    bnb_4bit_compute_dtype=torch.float16  # 연산은 float16으로 수행 (Mac 친화적)
)

# 3. 모델 로드
# device_map="auto"가 맥북의 메모리와 GPU(MPS) 상황에 맞춰 자동으로 배치합니다.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # 양자화 설정 적용
    trust_remote_code=True,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
generation_config = GenerationConfig.from_pretrained(model_name)

# 4. 프롬프트 준비
prompt = "KT에 대해 소개해줘"
messages = [
    {"role": "system", "content": "Mi:dm(믿:음)은 KT에서 개발한 AI 기반 어시스턴트이다."},
    {"role": "user", "content": prompt}
]

# 5. 입력 처리 (Mac GPU로 이동)
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)  # .to("cuda") 대신 model.device 사용

# 6. 생성
# 스트리머가 없으면 결과가 나올 때까지 기다려야 합니다.
output = model.generate(
    input_ids,
    generation_config=generation_config,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=256, # 답변이 잘리지 않게 조금 늘림
    do_sample=False,
)

# 7. 결과 출력
# 입력 프롬프트(input_ids) 길이를 제외하고 답변만 출력
decoded_output = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)
print(decoded_output)

🚀 모델 다운로드 및 로딩 중... (MLX 엔진)


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 159565.91it/s]



💬 답변 생성 중...



mx.metal.device_info is deprecated and will be removed in a future version. Use mx.device_info instead.
